In [ ]:
%load_ext dotenv
%dotenv

## Check experiment runs and get the best run

In [ ]:
import os
import mlflow
from mlflow.tracking import MlflowClient
from sklearn import datasets
import numpy as np

In [ ]:
# Create an instance of the MflowClient, we will use it to query our tracking server
client = MlflowClient()

In [ ]:
#Get the experiment
experiment = mlflow.get_experiment(experiment_id=1699708867784634)

In [ ]:
# Get experiment runs
runs = client.search_runs(experiment_ids = [experiment.experiment_id])

In [ ]:
#Choose a metric to get the best run
metrics = [run.data.metrics["mse"] for run in runs]
best_rmse = min(metrics)

best_run = [run for run in runs if run.data.metrics["mse"] == best_rmse][0]
best_run.info

<RunInfo: artifact_uri='dbfs:/databricks/mlflow-tracking/1699708867784634/3f07facfcf404bc8be7dd615cd26760d/artifacts', end_time=1671142573521, experiment_id='1699708867784634', lifecycle_stage='active', run_id='3f07facfcf404bc8be7dd615cd26760d', run_name='silent-panda-182', run_uuid='3f07facfcf404bc8be7dd615cd26760d', start_time=1671142568899, status='FINISHED', user_id=''>

In [ ]:
# Register the model from the best run
model_name = "my best model"

model_uri = f"runs:/{best_run.info.run_id}/model"
model_version = mlflow.register_model(model_uri = model_uri, name = model_name)

Registered model 'my best model' already exists. Creating a new version of this model...
2022/12/15 23:19:51 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: my best model, version 6
Created version '6' of model 'my best model'.


## Load the model

In [ ]:
model_info = client.get_registered_model(name = model_name)

In [ ]:
model_info

<RegisteredModel: creation_timestamp=1671142265785, description='', last_updated_timestamp=1671142791185, latest_versions=[<ModelVersion: creation_timestamp=1671142791185, current_stage='None', description='', last_updated_timestamp=1671142792170, name='my best model', run_id='3f07facfcf404bc8be7dd615cd26760d', run_link='', source='dbfs:/databricks/mlflow-tracking/1699708867784634/3f07facfcf404bc8be7dd615cd26760d/artifacts/model', status='READY', status_message='', tags={}, user_id='vitale@dataroots.io', version='6'>], name='my best model', tags={}>

In [ ]:
model = mlflow.sklearn.load_model(model_uri)

## Running inference

In [ ]:
diabetes = datasets.load_diabetes()
data = diabetes.data
preds = model.predict(data)

## Adding metadata to our model

In [ ]:
client.set_model_version_tag(model_name, model_version.version, key = "prediction_works", value = "true")

In [ ]:
model_version_info = client.get_model_version(model_name, model_version.version)

In [ ]:
model_version_info.tags

{'prediction_works': 'true'}

## Promoting our model to Staging

We are happy with our model (for now), so let's promote it to Staging

In [ ]:
client.transition_model_version_stage(model_name, model_version.version, stage = "Staging")

<ModelVersion: creation_timestamp=1671142791185, current_stage='Staging', description='', last_updated_timestamp=1671142968508, name='my best model', run_id='3f07facfcf404bc8be7dd615cd26760d', run_link='', source='dbfs:/databricks/mlflow-tracking/1699708867784634/3f07facfcf404bc8be7dd615cd26760d/artifacts/model', status='READY', status_message='', tags={}, user_id='3967460567028002', version='6'>

In [ ]:
model_version_info = client.get_model_version(model_name, model_version.version)

In [ ]:
# check if it worked
model_version_info.current_stage

'Staging'